#### Names of people in the group

Please write the names of the people in your group in the next cell.

Name of person A Karl Edvin Undheim

Name of person B

In [0]:
# We need to install 'ipython_unittest' to run unittests in a Jupyter notebook
!pip install -q ipython_unittest

You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.


In [0]:
# Loading modules that we need
import unittest
from pyspark.sql.dataframe import DataFrame
from typing import Any

In [0]:
# A helper function to load a table (stored in Parquet format) from DBFS as a Spark DataFrame 
def load_df(table_name: "name of the table to load") -> DataFrame:
    return spark.read.parquet(table_name)

users_df = load_df("/user/hive/warehouse/users")
comments_df = load_df("/user/hive/warehouse/comments")
posts_df = load_df("/user/hive/warehouse/posts")

#### Subtask 1: implenenting two helper functions
Impelment these two functions:
1. 'run_query' that gets a Spark SQL query and run it on df which is a Spark DataFrame; it returns the content of the first column of the first row of the DataFrame that is the output of the query;
2. 'run_query2' that is similar to 'run_query' but instead of one DataFrame gets two; it returns the content of the first column of the first row of the DataFrame that is the output of the query.

Note that the result of a Spark SQL query is itself a Spark DataFrame.

In [0]:
def run_query(query: "a SQL query string", df: "the DataFrame that the query will be executed on") -> Any:
  # tempview defines how to call the dataframe in the query
  df.createOrReplaceTempView("df");
  sqlDf = spark.sql(query)
  return sqlDf.first()[0]

def run_query2(query: "a SQL query string", df1: "DataFrame A", df2: "DataFrame B") -> Any:
  # Same as previous function but here we have two different dataframes
  df1.createOrReplaceTempView("df1");
  df2.createOrReplaceTempView("df2");
  sqlDf = spark.sql(query)
  return sqlDf.first()[0]

In [0]:
# Loading 'ipython_unittest' so we can use '%%unittest_main' magic command
%load_ext ipython_unittest

The ipython_unittest extension is already loaded. To reload it, use:
  %reload_ext ipython_unittest


#### Subtask 2: writing a few queries
Write the following queries in SQL to be executed by Spark in the next cell.

1. 'q1': find the 'Id' of the most recently created post ('df' is 'posts_df') 
2. 'q2': find the number users
3. 'q3': find the 'Id' of the user who posted most number of answers
4. 'q4': find the number of questions
5. 'q5': find the display name of the user who posted most number of comments

Note that 'q1' is already available below as an example. Moreover, remmebr that Spark supports ANSI SQL 2003 so your queries have to comply with that standard.

In [0]:
q1 = "SELECT * FROM df ORDER BY CreationDate DESC limit 1"

q2 = "SELECT COUNT(DISTINCT Id) FROM df"

q3 = "SELECT df.OwnerUserId, COUNT(df.PostTypeId) AS AnswerCount \
      FROM df \
      WHERE df.PostTypeId==2 \
      GROUP BY df.OwnerUserId \
      ORDER BY AnswerCount DESC"

q4 = "SELECT COUNT(PostTypeId) AS QuestionCount\
      FROM df\
      WHERE PostTypeId==1"

# Here I create a table containing each userId and how many comments this id has posted. This table must then be joined with the user table to be able to extract displayname.
q5 = "SELECT df1.DisplayName\
      FROM \
        (SELECT df2.UserId, COUNT(df2.UserId) AS AnswerCount \
        FROM df2 \
        GROUP BY df2.UserId \
        ORDER BY AnswerCount DESC \
        limit 1) C INNER JOIN df1 ON C.UserId==df1.Id"

#### Subtask 3: validating the implementations by running the tests

Run the cell below and make sure that all the tests run successfully.

In [0]:
%%unittest_main
class TestTask2(unittest.TestCase):
    def test_q1(self):
        # find the id of the most recent post
        r = run_query(q1, posts_df)
        self.assertEqual(r, 95045)

    def test_q2(self):
        # find the number of the users
        r = run_query(q2, users_df)
        self.assertEqual(r, 91616)

    def test_q3(self):
        # find the user id of the user who posted most number of answers
        r = run_query(q3, posts_df)
        self.assertEqual(r, 64377)

    def test_q4(self):
        # find the number of questions
        r = run_query(q4, posts_df)
        self.assertEqual(r, 28950)

    def test_q5(self):
        # find the display name of the user who posted most number of comments
        r = run_query2(q5, users_df, comments_df)
        self.assertEqual(r, "Neil Slater")

Success.....
----------------------------------------------------------------------
Ran 5 tests in 5.502s

OK
Out[14]: <unittest.runner.TextTestResult run=5 errors=0 failures=0>

#### Subtask 4: answering to questions about Spark related concepts

Please answer the following questions. Write your answer in one to two short paragraphs. Don't copy-paste; instead, write your own understanding.

1. What is the difference between 'DataFrame', 'Dataset', and 'Resilient Distributed Datasets (RDD)'? 
2. When do you suggest using RDDs instead of using DataFrames?
3. What is the main benefit of using DataSets instead of DataFrames?

Write your answers in the next cell.

1: Together they make up the three different spark APIs which can be used.
All of them are are immutable collections of objects which are partitioned across nodes in a cluster. They are all lazily evaluated and fault tolerant.
- The original API used in spark. The RDD(Resilient Distributed Datasets) is unstructured, meaning there is no inferred schema for columns and rows. It is also type safe, meaning validation of datatypes happens during compiling, rather than during execution. 
- The DataFrame expands on the RDD mainly by providing named columns(structured), and supports operations such as aggregate, select and sum. However it is no longer type safe.
- The Dataset expands on the dataframe and aims to combine parts from the RDD and dataframe. Notably type safety is provided aswell as many other features.

2: RDDs could be used when you are working with unstructured data and you dont need to use a schema. Additionally if you only require low level transformations they are useful, as you dont need the operations provided by dataframes/datasets. This allows the user more control.

3: The main benefit is that datasets support the object oriented interface and type safety used in the RDDs. This combines the best aspects of RDDs and dataframes.